In [9]:
import os
import numpy as np
import pandas as pd
import datetime as dt
import pytz
import time
import calendar
import numpy as np
from statistics import mean

from uce_resources import get_mms_data, get_applied_forecast, get_current_forecast

In [10]:
from settings.sites import ceg as sites_list

today = dt.datetime.today() #- dt.timedelta(days=1)

target_year = today.year
target_month = today.month
forecasts_types = ['real']

target_folder = 'data/results/{}-{:0>2}/'.format(target_year, target_month)
if not os.path.exists(target_folder):
    os.makedirs(target_folder)

# sites_list = ['Balivka', 'Solone', 'Stepnohirsk']

# Data preparation section

In [11]:
import importlib
imported_module = importlib.import_module("get_current_production_powermeter")
importlib.reload(imported_module)
from get_current_production_powermeter import OperativeProduction
from settings.apis import BORD_API_SETTINGS

PowermeterDataGetter = OperativeProduction(**BORD_API_SETTINGS)

Token updated!
Site ids updated!
Devices updated!
Devices updated!
Devices updated!
Devices updated!
Devices updated!
Devices updated!
Devices updated!
Devices updated!
Devices updated!
Devices updated!
Devices updated!
Devices updated!
Devices updated!
Devices updated!
Devices updated!
Devices updated!
Devices updated!
Devices updated!
Devices updated!
Devices updated!
Devices updated!
Devices updated!
Devices updated!
Devices updated!
Devices updated!
Devices updated!
Devices updated!
Devices updated!
Devices updated!
Devices updated!
Devices updated!
Devices updated!
Devices updated!
Devices updated!
Devices updated!
Devices updated!
Devices updated!
Devices updated!
Devices updated!
Devices updated!
Devices updated!
Devices updated!
Devices updated!
Devices updated!
Devices updated!
Devices updated!
Devices updated!


In [12]:
from sqlalchemy import create_engine, MetaData
from sqlalchemy.pool import NullPool
from sqlalchemy.sql import select, and_
from settings.db import DO_URL

engine_source = create_engine(DO_URL, poolclass=NullPool)
metadata_source = MetaData()
metadata_source.reflect(bind=engine_source)


from settings.db import WAREHOSUE_URL

engine_warehouse = create_engine(WAREHOSUE_URL, poolclass=NullPool)

C:\Users\n.kozelets\AppData\Local\Temp\ipykernel_13268\1008880204.py:8: SAWarning: Did not recognize type 'point' of column 'location'
  metadata_source.reflect(bind=engine_source)


In [13]:
sites_data = list()

with engine_source.connect() as connection:
        
    for site in sites_list:
        start = time.time()
        # print('-'*50)
        # print(site)
        site_data = dict()

        sites_table = metadata_source.tables['sites']
        list_to_select = [
            sites_table.c.id, 
            sites_table.c.legal_entity, 
            sites_table.c.location, 
            sites_table.c.region, 
            sites_table.c.cluster,
            sites_table.c.installed_capacity_dc,
            sites_table.c.grid_capacity,
            sites_table.c.w_code
            ]
        query = select(list_to_select).where(sites_table.c.displayable_name == site)
        site_id_response = connection.execute(query).fetchall()[0]

        site_id = site_id_response[0]
        legal_entity_id = site_id_response[1]
        location = site_id_response[2]
        region = site_id_response[3]
        cluster = site_id_response[4]
        capacity_dc = site_id_response[5]
        grid_capacity = site_id_response[6]
        w_code = site_id_response[7]
        
        with engine_warehouse.connect() as connection_warehouse:
            query = f"""
            SELECT grid_capacity from dim_site
            WHERE site_name = '{site}';
            """
            response = connection_warehouse.execute(query).fetchall()[0]
            
        grid_capacity = response[0]

        latitude, longitude = map(float, location.replace('(', '').replace(')', '').split(','))

        powermeter_data = PowermeterDataGetter.get_data(w_code, today)
        powermeter_data.columns= ['yield']
        # if powermeter_data is not None and len(powermeter_data) > 0:
        mms_data = powermeter_data
            #print(mms_data.loc[mms_data.index.intersection(inverters_data.index)])
            #raise KeyboardInterrupt

        first_date = today- dt.timedelta(days=1)
        last_date = dt.date(
            target_year, 
            target_month, 
            calendar.monthrange(target_year, target_month)[1]
        ) + dt.timedelta(days=7)
        # print(f"first_date: {first_date} last_date: {last_date}")

        applied_forecast = get_applied_forecast(
            site_id, 
            first_date,
            last_date,
            connection=connection, 
            db_table=metadata_source.tables['forecasting_data']
        )

        applied_forecast.columns = ['forecast']
        # print('Forecast data of | {} records |'.format(len(applied_forecast)))
        # print('applied_forecast.index.min:', applied_forecast.index.min())
        # print('applied_forecast.index.max:',applied_forecast.index.max())
        

        forecast = applied_forecast
        forecast_data = pd.concat([forecast, mms_data.loc[mms_data.index >= forecast.index.min()]], axis=1, join='outer').reindex(columns=['yield', 'forecast'])

        site_series = pd.Series(index=forecast_data.index, data=site)
        latitude_series = pd.Series(index=forecast_data.index, data=latitude)
        longitude_series = pd.Series(index=forecast_data.index, data=longitude)
        region_series = pd.Series(index=forecast_data.index, data=region)
        cluster_series = pd.Series(index=forecast_data.index, data=cluster)
        capacity_dc_series = pd.Series(index=forecast_data.index, data=capacity_dc)
        grid_capacity_series = pd.Series(index=forecast_data.index, data=grid_capacity)

        site_data = pd.concat([
                    site_series, 
                    latitude_series, 
                    longitude_series,
                    region_series, 
                    cluster_series,
                    capacity_dc_series,
                    grid_capacity_series
                ], 
                axis=1
            )
        site_data.columns = ['site', 'latitude', 'longitude', 'region', 'cluster', 'capacity_dc', 'grid_capacity']                       
        site_data['date'] = site_data.index.strftime('%Y-%m-%d')
        site_data['hour'] = site_data.index.hour + 1
        site_data['datetime'] = site_data.index.strftime('%Y-%m-%dT%H:%M')
        site_data['datetime_tz'] = site_data.index.tz_localize(pytz.utc).tz_convert(pytz.timezone('europe/kiev')).strftime('%Y-%m-%dT%H:%M%z')
        
        site_data = pd.concat([site_data, forecast_data], axis=1)

        site_data['error'] = site_data['yield'] - site_data['forecast']
        site_data['error_positive'] = site_data['error'].apply(lambda x: x * (x >= 0))
        site_data['error_negative'] = site_data['error'].apply(lambda x: x * (x < 0))
        site_data['error_abs'] = site_data['error'].apply(abs)
        site_data['error_type'] = site_data['error'].apply(lambda x: 'negative' if x < 0 else 'positive')
        
        sites_data.append(site_data)
        end = time.time()

        print('{}: ок! Processing took {} seconds'.format(site, round(end - start, 2)))

Data read successfully: 5, 922
Bar: ок! Processing took 1.28 seconds
Data read successfully: 6, 902
Verkhivka: ок! Processing took 0.83 seconds
Data read successfully: 7, 921
Balky: ок! Processing took 0.77 seconds
Data read successfully: 8, 712
Data read successfully: 8, 717
Sharhorod_1: ок! Processing took 1.41 seconds
Data read successfully: 10, 711
Chechelnyk_1: ок! Processing took 1.07 seconds
Data read successfully: 11, 747
Chechelnyk_2: ок! Processing took 0.84 seconds
Data read successfully: 12, 953
Stanislavchyk: ок! Processing took 0.8 seconds
Data read successfully: 13, 939
Data read successfully: 13, 940
Kopaihorod: ок! Processing took 1.24 seconds
Data read successfully: 14, 573
Data read successfully: 14, 575
Cherniatka: ок! Processing took 1.25 seconds
Data read successfully: 16, 794
Hlybochok_1: ок! Processing took 0.85 seconds
Data read successfully: 17, 796
Hlybochok_2.1: ок! Processing took 0.89 seconds
Data read successfully: 18, 790
Data read successfully: 18, 791


In [14]:
data = pd.concat(sites_data, ignore_index=True).fillna(0)
data = data.drop_duplicates(keep='first')
data.to_csv(target_folder + 'mart_operative_forecasting_results.csv', index=False)

In [15]:
data_26 = data.copy()
# data_26 = data.loc[data.date == '2022-03-26']
site = 'Myroliubivka'
print(data_26.loc[(data_26.site == site) & (data_26['yield'] <= 0) & (data_26['forecast'] < 0)])
data_26.loc[(data_26.site == site) & (data_26['yield'] <= 0) & (data_26['forecast'] < 0)][['yield', 'forecast']].mean()

              site   latitude  longitude region  cluster  capacity_dc  \
2688  Myroliubivka  46.726252  32.286332     KH        5        11477   
2689  Myroliubivka  46.726252  32.286332     KH        5        11477   
2690  Myroliubivka  46.726252  32.286332     KH        5        11477   
2691  Myroliubivka  46.726252  32.286332     KH        5        11477   
2692  Myroliubivka  46.726252  32.286332     KH        5        11477   
2709  Myroliubivka  46.726252  32.286332     KH        5        11477   
2710  Myroliubivka  46.726252  32.286332     KH        5        11477   
2711  Myroliubivka  46.726252  32.286332     KH        5        11477   
2712  Myroliubivka  46.726252  32.286332     KH        5        11477   
2713  Myroliubivka  46.726252  32.286332     KH        5        11477   
2714  Myroliubivka  46.726252  32.286332     KH        5        11477   
2715  Myroliubivka  46.726252  32.286332     KH        5        11477   
2716  Myroliubivka  46.726252  32.286332     KH    

yield       -1.947368
forecast   -23.000000
dtype: float64

### Inserting data to data mart

In [16]:
df_full = data.copy()
tuples_full = [tuple(x) for x in df_full.to_numpy()]
# print(tuples_full[-2:])
columns = list(df_full.columns)

columns_unique = [columns[0], *columns[5:7]]
# print(columns_unique)

df_update = df_full.copy().drop(columns=columns_unique)
tuples_update = [tuple(x) for x in df_update.to_numpy()]
columns_update = list(df_update.columns)
# print(columns_update)
# SQL query to execute
query_1 = 'INSERT INTO mart_operative_forecasting_result({}) VALUES {}'.format(','.join(columns), str(tuples_full).replace('[', '').replace(']', ''))
query_2 = '''
ON CONFLICT (site, date, hour) 
DO UPDATE SET
latitude = excluded.latitude,
longitude = excluded.longitude,
region = excluded.region,
cluster = excluded.cluster,
datetime = excluded.datetime,
datetime_tz = excluded.datetime_tz,
yield = excluded.yield,
forecast = excluded.forecast,
error = excluded.error,
error_positive = excluded.error_positive,
error_negative = excluded.error_negative,
error_abs = excluded.error_abs,
error_type = excluded.error_type,
capacity_dc = excluded.capacity_dc,
grid_capacity = excluded.grid_capacity;'''
# print(query_1, query_2)
with engine_warehouse.connect() as connection:
    connection.execute(query_1 + '\n' + query_2)
